# Computing the initial stability and SASA scores

After downloading and repairing the proteins (check `./downloading_the_data.ipynb`), let's compute and save the initial stability and SASA scores. The output of this notebook is a `csv` file with columns `["pdb_id", "stability", "sasa", "pH"]`

In [1]:
from pathlib import Path
import json

from poli.core.util.proteins.foldx import FoldxInterface

THIS_DIR = Path().resolve()
working_dir_base = THIS_DIR / "repaired_pdbs"

# Load the pH values
with open(THIS_DIR / "pHs.json", "r") as f:
    pHs = json.load(f)

In [3]:
foldx = FoldxInterface(working_dir_base)

rows = []
for folder in working_dir_base.iterdir():
    row = {}
    if folder.is_dir():
        # Set the interface to work in the folder
        foldx.working_dir = folder

        # Get the name of the pdb from the folder name
        pdb_name_and_chain = folder.name
        pdb_id, chain = pdb_name_and_chain.split("_")
        row["pdb_id"] = pdb_id
        row["pH"] = pHs[pdb_id.upper()]

        # Compute the stability and SASA
        stability, sasa = foldx.compute_stability_and_sasa(
            pdb_file=folder / f"{pdb_id}_{chain}_Repair.pdb",
            mutations=None,
        )

        row["stability"] = stability
        row["sasa"] = sasa

        rows.append(row)

   ********************************************
   ***                                      ***
   ***             FoldX 4 (c)              ***
   ***                                      ***
   ***     code by the FoldX Consortium     ***
   ***                                      ***
   ***     Jesper Borg, Frederic Rousseau   ***
   ***    Joost Schymkowitz, Luis Serrano   ***
   ***    Peter Vanhee, Erik Verschueren    ***
   ***     Lies Baeten, Javier Delgado      ***
   ***       and Francois Stricher          ***
   *** and any other of the 9! permutations ***
   ***   based on an original concept by    ***
   ***   Raphael Guerois and Luis Serrano   ***
   ********************************************

1 models read: 4hq8_B_Repair.pdb

BackHbond       =               -163.88
SideHbond       =               -71.95
Energy_VdW      =               -257.74
Electro         =               -12.97
Energy_SolvP    =               328.24
Energy_SolvH    =               -341.60
Energy_v

Let's change the `None`s for `NaN`s

In [4]:
for row in rows:
    if row["pH"] is None:
        row["pH"] = float("nan")

In [5]:
rows

[{'pdb_id': '4hq8',
  'pH': 7.5,
  'stability': 37.6997,
  'sasa': 10590.687610794252},
 {'pdb_id': '6dej',
  'pH': nan,
  'stability': 13.5624,
  'sasa': 11102.123802587561},
 {'pdb_id': '4h3l',
  'pH': 6.5,
  'stability': 25.6074,
  'sasa': 10051.45979711304},
 {'pdb_id': '4eds',
  'pH': 7.0,
  'stability': 27.3628,
  'sasa': 10300.85007823728},
 {'pdb_id': '6mgh',
  'pH': 4.0,
  'stability': 35.9202,
  'sasa': 8224.213615811059},
 {'pdb_id': '2vad',
  'pH': 7.6,
  'stability': 69.8402,
  'sasa': 10377.771961794057},
 {'pdb_id': '4jge',
  'pH': 7.0,
  'stability': 27.3797,
  'sasa': 10215.967524560014},
 {'pdb_id': '5jva',
  'pH': 8.5,
  'stability': 20.1138,
  'sasa': 10901.775592343227},
 {'pdb_id': '4edo',
  'pH': 7.0,
  'stability': 26.4975,
  'sasa': 10260.716898105311},
 {'pdb_id': '3rwa',
  'pH': 7.5,
  'stability': 11.6063,
  'sasa': 11660.478524556138},
 {'pdb_id': '3nt9',
  'pH': 5.6,
  'stability': 9.23374,
  'sasa': 11083.908044134329},
 {'pdb_id': '4nws',
  'pH': 3.5,
  

In [6]:
import pandas as pd

df = pd.DataFrame(rows)

In [7]:
df.head()

,pdb_id,pH,stability,sasa
0,4hq8,7.5,37.6997,10590.687611
1,6dej,NaN,13.5624,11102.123803
2,4h3l,6.5,25.6074,10051.459797
3,4eds,7.0,27.3628,10300.850078
4,6mgh,4.0,35.9202,8224.213616


Let's save it, after appending the title it has in `rcsb`:

In [10]:
import pypdb

df["title"] = df["pdb_id"].apply(lambda pdb_id: pypdb.get_info(pdb_id)["struct"]["title"])

df.to_csv(THIS_DIR / "initial_stability_and_sasa.csv", index=False)

## What's the current pareto front?

In [4]:
from pathlib import Path
import pandas as pd

THIS_DIR = Path().resolve()

df = pd.read_csv(THIS_DIR / "initial_stability_and_sasa.csv")

Let's do a scatter plot of these stabilities and SASA scores

In [8]:
# import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np

import plotly.express as px

import torch

from botorch.utils.multi_objective import pareto

# sns.set_theme(style="whitegrid")


data = df[["sasa", "stability"]].to_numpy()
pareto_mask = pareto.is_non_dominated(torch.from_numpy(data))

df_pareto = df.drop(np.where(~pareto_mask)[0])

fig = px.scatter(df, x="sasa", y="stability", hover_data=["pdb_id", "title"])
fig.show()

print("Pareto front:")
print(df_pareto["title"].values)

Pareto front:
['Monomeric red fluorescent protein, DsRed.M1'
 'Crystal structure of circular-permutated mKate'
 'Crystal structure of mEos2-A69T fluorescent protein'
 'Crystal structure of LSSmScarlet - a genetically encoded red fluorescent protein with a large Stokes shift'
 'Crystal structure of mCherry'
 'Fast maturing red fluorescent protein, DsRed.T4'
 'Fluorescent protein from Acropora digitifera'
 'Structure of the Red Fluorescent Protein mScarlet at pH 7.8']


## Saving the pareto front for running `nsga-ii`

In [10]:
df_pareto.to_csv(THIS_DIR / "initial_pareto_front.csv", index=False)